# Pandas Style

For reference: https://www.stickeria.com.br/img/products/panda-style_1_1200.jpg

In [1]:
# This lets the notebook import the "cosmic_rAI" module
import sys
sys.path.insert(0, '..')

import math
import itertools
import numpy as np
import pandas as pd

from cosmic_rAI import data_prep

In [2]:
mat1 = np.load('../data/sim_12360_00.npy').item()  # protons
mat2 = np.load('../data/sim_12362_00.npy').item()  # iron

## I. Making DataFrames from Arrays

### (a) Event DataFrame

In [3]:
def partition_list(lst, x_name='x', y_name='y'):
    """Given list that alternates between x,y values,
    partition into dictionary"""
    
    return {
        x_name: lst[::2],
        y_name: lst[1::2]}

In [416]:
def event_df_from_matrix(mat):
    """Creates multi-index DataFrame from select attrs in matrix"""
    frames = {
        'charges': pd.DataFrame(mat['Charges']),
        'energy': pd.DataFrame(mat['Energy']),
        'composition': pd.DataFrame(mat['Composition']),
        'fit_status': pd.DataFrame(mat['Fit_status']),
        'core_MC': pd.DataFrame(partition_list(mat['core_MC'], 'x', 'y')),
        'core_reco': pd.DataFrame(partition_list(mat['core_reco'], 'x', 'y')),
        'dir_MC': pd.DataFrame(partition_list(mat['dir_MC'], 'azimuth', 'zenith')),
        'dir_reco': pd.DataFrame(partition_list(mat['dir_reco'], 'azimuth', 'zenith'))}
    return pd.concat(frames, axis=1).fillna(0)

In [5]:
def remove_nan_events(df, matrices):
    """Eliminates from dataframe any events (rows)
    where a sensor recorded NaN"""
    
    charges = list(itertools.chain.from_iterable(
        (m['Charges'] for m in matrices)))
    
    evil_indices = []
    for idx, event in enumerate(charges):
        if any(math.isnan(v) for k, v in event.items()):
            evil_indices.append(idx)
            
    return df.drop(evil_indices)

In [418]:
matrices = [mat1, mat2]

_df = pd.concat(map(event_df_from_matrix, matrices))
event_df = remove_nan_events(_df, matrices)

In [408]:
print("Old df:", len(_df))
print("New df:", len(event_df))

Old df: 31620
New df: 31120


In [419]:
event_df.tail(n=3)

charges                                                 ...      \
         0161 0162 0163 0164 0261 0262 0263 0264 0361 0362    ...       
15086     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       
15087     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       
15088     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    ...       

          core_MC               core_reco                dir_MC            \
                x           y           x           y   azimuth    zenith   
15086  -54.849927   24.516840  -81.159385   -4.965321  0.989706  0.534672   
15087  241.163034 -242.219720  254.189208 -276.710978  0.989706  0.534672   
15088 -281.788800  297.279114 -282.388461  292.040085  0.989706  0.534672   

       dir_reco                  energy fit_status  
        azimuth    zenith             0          0  
15086  0.989350  0.537727  7.999586e+06         OK  
15087  0.992155  0.527354  7.999586e+06         OK  
15088  0.989696  0.570790  7.999586e+06         OK  

[3 rows x 334 columns]

#### Another format...

In [10]:
def flatten(df):
    new_df = df.copy()
    lvl0 = df.columns.get_level_values(0).astype('str')
    lvl1 = df.columns.get_level_values(1).astype('str')
    cols = lvl0 + '_' +lvl1
    new_df.columns = cols
    return new_df

In [421]:
flat_df = flatten(event_df)
flat_df.head(n=3)

,charges_0161,charges_0162,charges_0163,charges_0164,charges_0261,charges_0262,charges_0263,charges_0264,charges_0361,charges_0362,...,core_MC_x,core_MC_y,core_reco_x,core_reco_y,dir_MC_azimuth,dir_MC_zenith,dir_reco_azimuth,dir_reco_zenith,energy_0,fit_status_0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,39.491699,27.012478,48.792127,25.608482,0.216553,0.765261,0.253442,0.899646,137553.657022,OK
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-249.483765,269.715753,-260.517562,262.071308,0.216553,0.765261,0.231127,0.715951,137553.657022,OK
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,76.900929,23.952042,79.645692,32.718007,0.355619,1.739962,0.373549,1.585760,220232.485147,OK


In [550]:
lvl0 = event_df.columns.get_level_values(0).astype('str')
lvl1 = event_df.columns.get_level_values(1).astype('str')
cols = lvl0 + '_' +lvl1

### (b) Sensor DataFrame

In [17]:
def sensor_df_from_matrix(mat):
    vals = {
        'gain': mat['Gain'][0],
        'pos_x': mat['Position'][0],
        'pos_y': mat['Position'][1],}
    return pd.DataFrame(vals)

In [218]:
sensor_df = sensor_df_from_matrix(mat1)  # mat1 and mat2 are eqv
len(sensor_df)

323

In [228]:
sensor_df.loc['3861':'4064']

,gain,pos_x,pos_y
3861,High,316.035004,-3.805000
3862,Low,316.035004,-3.805000
3863,High,311.074997,-12.415000
3864,Low,311.074997,-12.415000
3962,High,434.970001,3.940000
3963,High,429.565002,-4.585000
3964,Low,429.565002,-4.585000
4061,High,530.605011,20.969999
4062,Low,530.605011,20.969999
4063,High,522.024994,25.590000


### (c) Example Usage: Getting low gain sensors

In [20]:
charges_df = event_df['charges']
charges_df.head(n=3)

,0161,0162,0163,0164,0261,0262,0263,0264,0361,0362,...,7963,7964,8061,8062,8063,8064,8161,8162,8163,8164
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,1.543506,0.0,2.168716,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.439023,0.0,0.496192,0.0,7.982038,0.0,9.587134,0.0


In [24]:
high_gain_sensors = sensor_df.query("gain == 'High'").T
high_gain_sensors.head()

,0161,0163,0261,0263,0361,0363,0461,0463,0561,0563,...,7761,7763,7861,7863,7961,7963,8061,8063,8161,8163
gain,High,High,High,High,High,High,High,High,High,High,...,High,High,High,High,High,High,High,High,High,High
pos_x,-265.53,-255.7,-140.36,-130.635,-27.72,-20.395,105.655,115.265,214.74,219.905,...,-87.705,-97.1,2.11,-2.02,18.1,10.88,76.425,85.645,77.41,87.19
pos_y,-497.895,-496.07,-477.765,-476.575,-464.49,-458.975,-438.705,-436.865,-432.41,-424.685,...,468.725,465.195,494.62,503.595,-94.63,-87.695,-42.48,-46,37.615,39.84


In [25]:
low_gain_sensors = sensor_df.query("gain == 'Low'").T
low_gain_sensors.head()

,0162,0164,0262,0264,0362,0364,0462,0464,0562,0564,...,7762,7764,7862,7864,7962,7964,8062,8064,8162,8164
gain,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low,...,Low,Low,Low,Low,Low,Low,Low,Low,Low,Low
pos_x,-265.53,-255.7,-140.36,-130.635,-27.72,-20.395,105.655,115.265,214.74,219.905,...,-87.705,-97.1,2.11,-2.02,18.1,10.88,76.425,85.645,77.41,87.19
pos_y,-497.895,-496.07,-477.765,-476.575,-464.49,-458.975,-438.705,-436.865,-432.41,-424.685,...,468.725,465.195,494.62,503.595,-94.63,-87.695,-42.48,-46,37.615,39.84


In [203]:
sensor_df.loc['3861':'4064']

,gain,pos_x,pos_y
3861,High,316.035004,-3.805000
3862,Low,316.035004,-3.805000
3863,High,311.074997,-12.415000
3864,Low,311.074997,-12.415000
3962,High,434.970001,3.940000
3963,High,429.565002,-4.585000
3964,Low,429.565002,-4.585000
4061,High,530.605011,20.969999
4062,Low,530.605011,20.969999
4063,High,522.024994,25.590000


In [26]:
low_gain_df = charges_df.drop(high_gain_sensors.columns, axis=1)
low_gain_df.head(n=3)

,0162,0164,0262,0264,0362,0364,0462,0464,0562,0564,...,7762,7764,7862,7864,7962,7964,8062,8064,8162,8164
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [260]:
high_gain_df = charges_df.drop(low_gain_sensors.columns, axis=1)
high_gain_df = high_gain_df.drop('3962', axis=1)
high_gain_df.head(n=3)

,0161,0163,0261,0263,0361,0363,0461,0463,0561,0563,...,7761,7763,7861,7863,7961,7963,8061,8063,8161,8163
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.543506,2.168716
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.439023,0.496192,7.982038,9.587134


In [552]:
max(flat_df['dir_MC_zenith'])
max(dataframe['dir_MC_zenith'])

6.282403500872203

In [422]:
flat_df.columns[323:334]

Index(['composition_0', 'core_MC_x', 'core_MC_y', 'core_reco_x', 'core_reco_y',
       'dir_MC_azimuth', 'dir_MC_zenith', 'dir_reco_azimuth',
       'dir_reco_zenith', 'energy_0', 'fit_status_0'],
      dtype='object')

In [537]:
len(high_gain_df)

31120

In [541]:
# Create Pandas Dataframe of only events that are fit status OK 
# and combine high/low gain (if there's any value on low gain, use that instead of high)

# Is pretty slow

df = []
keys = high_gain_df.keys()
keys = keys.append(flat_df.columns[323:334])

# for j in range(100):
for j in range(len(high_gain_df)):
    if flat_df.iloc[j]['fit_status_0'] == 'OK':
        cur_series = []
        
        if (j+1)%1000 == 0:
            print(j+1)
        
        for i in range(len(high_gain_df.iloc[j])):
            if low_gain_df.iloc[j][i]:
                cur_series.append(low_gain_df.iloc[j].iat[i])
            else:
                cur_series.append(high_gain_df.iloc[j].iat[i])
                
        if any(charge >= 6 for charge in cur_series):
            [cur_series.append(item) for item in flat_df.iloc[j][flat_df.columns[323:334]].values]
            df.append(pd.Series(cur_series, keys))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000


In [542]:
dataframe = pd.concat(df, axis=1).T

In [520]:
dataframe.columns[161:]

Index(['composition_0', 'core_MC_x', 'core_MC_y', 'core_reco_x', 'core_reco_y',
       'dir_MC_azimuth', 'dir_MC_zenith', 'dir_reco_azimuth',
       'dir_reco_zenith', 'energy_0', 'fit_status_0'],
      dtype='object')

In [546]:
dataframe.to_pickle("dataframe.pkl")

In [553]:
df = pd.read_pickle("CleanedDataFrame.pkl")

In [555]:
df.head()

,0161,0163,0261,0263,0361,0363,0461,0463,0561,0563,...,core_MC_x,core_MC_y,core_reco_x,core_reco_y,dir_MC_azimuth,dir_MC_zenith,dir_reco_azimuth,dir_reco_zenith,energy_0,fit_status_0
0,0,0,0,0,0,0,0,0,0,0,...,76.9009,23.952,79.6457,32.718,0.355619,1.73996,0.373549,1.58576,220232,OK
1,0,0,0,0,0,0,0,0,0,0,...,-1.52907,-293.965,6.2853,-273.257,0.318676,1.89113,0.309681,1.97661,574079,OK
2,0,0,0,0,0,0,0,0,0,0,...,66.7311,-192.139,70.1029,-188.272,0.318676,1.89113,0.252595,1.9182,574079,OK
3,0,0,0,0,0,0,0,0,0,0,...,-141.046,-64.1063,-148.585,-44.9088,0.318676,1.89113,0.3283,1.7938,574079,OK
4,0,0,0,0,0,0,0,0,0,0,...,94.2443,-33.2704,94.2749,-44.9995,0.318676,1.89113,0.322193,1.87777,574079,OK
